In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv')
data

,vegetable,year,month,title,contents,value,value2
0,양파,1990,1,춘파양파 계약재배 당부,재배면적 감소로 인한 생산성을 높일것을 강조,0.30,0.0
1,양파,1990,2,폭설불구 서울지역 생필품값은 안정세,가격 안정세,0.70,1.0
2,마늘,1990,2,폭설불구 서울지역 생필품값은 안정세,가격 안정세,0.70,1.0
3,양파,1990,2,농진청 습해방지요령 발표,농작물 피해 최소화 바람,0.40,0.0
4,마늘,1990,2,농진청 습해방지요령 발표,농작물 피해 최소화 바람,0.40,0.0
...,...,...,...,...,...,...,...
2954,양파,2009,12,제주 17일부터 한파…평년대비 기온 3~6도 ↓,때문에 환기를 철저히 하고 병 발생 위험 작물은 살균제 농약을 살포해야 한다. 마늘...,0.21,NaN
2955,마늘,2009,12,경남도 창녕 우포늪에 생명환경농업단지 조성,"현재 우포늪 주변에는 6개 읍면, 1287농가가 벼와 양파, 마늘 등을 재배하고 있...",0.71,NaN
2956,양파,2009,12,경남도 창녕 우포늪에 생명환경농업단지 조성,"현재 우포늪 주변에는 6개 읍면, 1287농가가 벼와 양파, 마늘 등을 재배하고 있...",0.71,NaN
2957,마늘,2009,12,“농작물 재해 한시름 덜겠네”,"보험 도입 초기 재해피해 위험률이 높은 사과, 배 등 과수 7개 품목을 대상으로 시...",0.68,NaN


In [44]:
data = np.array(data)
print(data[0,6])

0.0


In [45]:
for i in range(len(data)):
  if data[i,5] >= 0.5:
    data[i,6] = 1
  else:
    data[i,6] = 0

In [4]:
onion_data = data[data['vegetable'] == '양파']
garlic_data = data[data['vegetable'] == '마늘']

IndexError: ignored

In [5]:
onion_data

NameError: ignored

In [ ]:
garlic_data

,vegetable,year,month,day,title,contents,value,value2
2,마늘,1990,2,2,폭설불구 서울지역 생필품값은 안정세,가격 안정세,0.70,1
4,마늘,1990,2,19,농진청 습해방지요령 발표,농작물 피해 최소화 바람,0.40,0
6,마늘,1990,2,26,장바구니 물가 한달 새 최소 64.5%까지 올라.,농산물 가격 급등에 따른 기사,0.20,0
8,마늘,1990,3,6,마늘 양파 출하 약정제 실시,재배면적에 따른 가격 보상정책,0.80,1
10,마늘,1990,3,13,전업농에 부재지주 땅 매입 자금 지원,가격에 따른 농업 제도 변경,0.74,1
...,...,...,...,...,...,...,...,...
901,마늘,1994,12,19,농림축산물 1백11개 품목에 특별긴급관세,"또한 현행 세율보다 높은 세율로 한도양허(CE)한 참깨, 마늘, 양파, 고추, 대추...",0.30,0
903,마늘,1994,12,22,光州지역 김장채소값 오름세,"또 마늘은 ㎏당 3천원에서 4천원으로 1천원이 올랐고 생강도 3천5백원에서 4천원,...",0.19,0
905,마늘,1994,12,24,개방대응 농수산물 수입관리 강화,"정부는 또 내년에 농안기금 1천8백96억원을 들여 콩, 팥, 녹두, 참깨, 땅콩, ...",0.43,0
907,마늘,1994,12,27,내년 초 마늘.양파 수입으로 피해 우려,(濟州=聯合) 洪貞杓 기자 = 내년부터 마늘과 양파 수입이 허용돼 제주도내 관련농가...,0.33,0


In [ ]:
onion_data = np.array(onion_data)
garlic_data = np.array(garlic_data)

In [46]:
onion_train = data[:,4]
onion_target = data[:,6]

In [47]:
print(onion_train[0])

재배면적 감소로 인한 생산성을 높일것을 강조


In [48]:
print(onion_train.shape,onion_target.shape)

(2959,) (2959,)


In [49]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [50]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [51]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

In [15]:
!pip install kss

     |████████████████████████████████| 71kB 5.0MB/s 


In [52]:
import kss

sentence_tokenized_text = []
for i, line in enumerate(onion_train):
    line = line.strip()
    for sent in kss.split_sentences(line):
        sentence_tokenized_text.append(sent.strip())

In [53]:
cleaned_corpus = []
for sent in sentence_tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [54]:
len(cleaned_corpus)

5071

In [55]:
for i in range(0, 10):
    print(cleaned_corpus[i])

재배면적 감소로 인한 생산성을 높일것을 강조
가격 안정세
가격 안정세
농작물 피해 최소화 바람
농작물 피해 최소화 바람
농산물 가격 급등에 따른 기사
농산물 가격 급등에 따른 기사
재배면적에 따른 가격 보상정책
재배면적에 따른 가격 보상정책
가격에 따른 농업 제도 변경


In [20]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [56]:
basic_preprocessed_corpus = clean_text(cleaned_corpus)

In [57]:
len(basic_preprocessed_corpus)

5071

In [58]:
for i in range(0, 10):
    print(basic_preprocessed_corpus[i])

재배면적 감소로 인한 생산성을 높일것을 강조
가격 안정세
가격 안정세
농작물 피해 최소화 바람
농작물 피해 최소화 바람
농산물 가격 급등에 따른 기사
농산물 가격 급등에 따른 기사
재배면적에 따른 가격 보상정책
재배면적에 따른 가격 보상정책
가격에 따른 농업 제도 변경


In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [60]:
data2 = np.concatenate((np.array(basic_preprocessed_corpus[:2959]).reshape(-1,1),onion_target.reshape(-1,1)),axis=1)

In [61]:
data2.shape

(2959, 2)

In [62]:
train_data, test_data = train_test_split(data2, test_size = 0.1, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

훈련용 리뷰의 개수 : 2663
테스트용 리뷰의 개수 : 296


In [ ]:
mecab = Mecab()
print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))

['와', '이런', '것', '도', '상품', '이', '라고', '차라리', '내', '가', '만드', '는', '게', '나을', '뻔']


In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [63]:
train_x = train_data[:,0]
train_y = train_data[:,1]
test_x = test_data[:,0]
test_y = test_data[:,1]

In [30]:
print(train_x[0])

그밖에 복숭아통조림 , 종자용옥수수 등은 년 내에 , 당류 . 초콜릿 . 면류 . . . .


In [31]:
vocab_size = 20000
embedding_dim = 16
max_length = 200
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

word_index = tokenizer.word_index
print(word_index)
training_sequences = tokenizer.texts_to_sequences(train_x)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(training_sequences)
testing_sequences = tokenizer.texts_to_sequences(test_x)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

{'<OOV>': 1, '마늘': 2, '양파': 3, '등': 4, '일': 5, '고추': 6, '聯合': 7, '있다': 8, '따르면': 9, '개': 10, '것으로': 11, '마늘과': 12, '또': 13, '배추': 14, '있는': 15, '서울': 16, '보리': 17, '경우': 18, "'": 19, '가격이': 20, '양파는': 21, '위해': 22, '지난해': 23, '올해': 24, '양파의': 25, '지난': 26, '등의': 27, '이': 28, '사과': 29, '비해': 30, '무': 31, '및': 32, '배': 33, '크게': 34, '쇠고기': 35, '농산물': 36, '감자': 37, '각각': 38, '마늘은': 39, '쌀': 40, '참깨': 41, '큰': 42, '돼지고기': 43, '㎏당': 44, '했다': 45, '따라': 46, '특히': 47, '감귤': 48, '한편': 49, '도내': 50, '콩': 51, '│': 52, '가': 53, '㎏': 54, '현재': 55, '파': 56, '정부는': 57, '가격': 58, '수': 59, '년': 60, '최근': 61, '등을': 62, '양파등': 63, '그러나': 64, '관계자는': 65, '에': 66, '주요': 67, '비가': 68, '양파와': 69, '고구마': 70, '이밖에': 71, '이번': 72, '농림수산부는': 73, '상품': 74, '천원으로': 75, '가운데': 76, '결과': 77, '우려되고': 78, '백': 79, '농협': 80, '이들': 81, '당근': 82, '오는': 83, '품목은': 84, '옥수수': 85, '대상으로': 86, '닭고기': 87, '光州': 88, '양배추': 89, '반면': 90, '천백원으로': 91, '오이': 92, '오른': 93, '높은': 94, '생강': 95, '재배면적이': 96, '통해': 97, '가뭄으로': 98, '정

In [64]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(train_y).astype(np.float32)
print(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(test_y).astype(np.float32)

[0. 1. 0. ... 1. 0. 0.]


In [65]:
from tensorflow.keras.layers import Embedding, Dense, GRU,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 200))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256,return_sequences=True))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         4000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 256)         467968    
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dense_6 (Dense)              (None, None, 128)         32896     
_________________________________________________________________
dense_7 (Dense)              (None, None, 128)         16512     
_________________________________________________________________
dense_8 (Dense)              (None, None, 1)          

In [34]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(training_padded,training_labels, epochs=100, callbacks=[mc],validation_data=(testing_padded,testing_labels))

Epoch 1/100
84/84 [==============================] - 12s 92ms/step - loss: 0.6944 - acc: 0.4868 - val_loss: 0.6932 - val_acc: 0.4899

Epoch 00001: val_loss did not improve from 0.04466
Epoch 2/100
84/84 [==============================] - 7s 80ms/step - loss: 0.6935 - acc: 0.5004 - val_loss: 0.6930 - val_acc: 0.5101

Epoch 00002: val_loss did not improve from 0.04466
Epoch 3/100
84/84 [==============================] - 7s 79ms/step - loss: 0.6936 - acc: 0.4959 - val_loss: 0.6930 - val_acc: 0.5072

Epoch 00003: val_loss did not improve from 0.04466
Epoch 4/100
84/84 [==============================] - 7s 78ms/step - loss: 0.6937 - acc: 0.4920 - val_loss: 0.6930 - val_acc: 0.5101

Epoch 00004: val_loss did not improve from 0.04466
Epoch 5/100
84/84 [==============================] - 7s 79ms/step - loss: 0.6931 - acc: 0.5108 - val_loss: 0.6936 - val_acc: 0.4914

Epoch 00005: val_loss did not improve from 0.04466
Epoch 6/100
84/84 [==============================] - 7s 79ms/step - loss: 0.657

In [38]:
a = model.predict(testing_padded)
a.shape
print(np.mean(a[3]))

0.7529307


In [39]:
testing_labels

array([0.63, 0.66, 0.41, 0.21, 0.44, 0.88, 0.61, 0.08, 0.56, 0.81, 0.58,
       0.68, 0.71, 0.8 , 0.39, 0.66, 0.91, 0.77, 0.14, 0.13, 0.38, 0.47,
       0.3 , 0.61, 0.28, 0.58, 0.63, 0.43, 0.28, 0.66, 0.32, 0.78, 0.87,
       0.35, 0.63, 0.64, 0.5 , 0.19, 0.17, 0.23, 0.37, 0.68, 0.17, 0.74,
       0.24, 0.25, 0.28, 0.71, 0.27, 0.38, 0.79, 0.64, 0.63, 0.7 , 0.64,
       0.43, 0.36, 0.81, 0.77, 0.87, 0.64, 0.28, 0.67, 0.4 , 0.48, 0.71,
       0.17, 0.8 , 0.17, 0.76, 0.78, 0.16, 0.81, 0.63, 0.64, 0.77, 0.23,
       0.31, 0.66, 0.74, 0.67, 0.63, 0.66, 0.23, 0.24, 0.57, 0.62, 0.29,
       0.36, 0.34, 0.16, 0.6 , 0.22, 0.21, 0.14, 0.14, 0.31, 0.36, 0.23,
       0.46, 0.84, 0.18, 0.23, 0.61, 0.44, 0.28, 0.58, 0.71, 0.71, 0.33,
       0.36, 0.61, 0.48, 0.24, 0.63, 0.67, 0.56, 0.11, 0.66, 0.63, 0.69,
       0.34, 0.21, 0.19, 0.43, 0.32, 0.87, 0.57, 0.65, 0.31, 0.51, 0.38,
       0.79, 0.69, 0.28, 0.7 , 0.36, 0.7 , 0.67, 0.59, 0.24, 0.31, 0.54,
       0.59, 0.17, 0.58, 0.31, 0.74, 0.33, 0.67, 0.